In [ ]:
import os
import re
import hashlib
import pandas as pd
from docx import Document
import ftfy 

In [1]:
# getting full text labels
# loading data
folder_path = r"C:\Users\Kvrgic\Desktop\Soriano\Soriano_texts"  
output_csv_path = "fulltext_sentences_labeled.csv"

# cleaning data
def clean_text(text):
    return ftfy.fix_text(text).strip()
# normalizing data
def get_hash(text):
    normalized = text.lower().strip()
    normalized = " ".join(normalized.split())
    return hashlib.md5(normalized.encode("utf-8")).hexdigest()
# cleaning data
def simple_sent_tokenize(text):
    parts = re.split(r'(?<=[\.\!\?])\s+', text)
    return [p.strip() for p in parts if p.strip()]
    
# regex for direct speech
direct_speech_patterns = [
    # core direct speech 
    r"\bmi disse\b", r"\bmi rispose\b", r"\bmi replicò\b", r"\bmi dichiarò\b", r"\bmi aggiunse\b", r"\bmi promise\b",
    r"\bet ella\b.*?\bdisse\b", r"\bet ella\b.*?\brispose\b",
    r"\ba me\b.*?\bdisse\b", r"\ba me\b.*?\brispose\b", r"\ba me\b.*?\bparlò\b",
    r"\busc[iì]\b.*?\ba dirmi\b",

    # gerunds and participial phrases 
    r"\bdicendo\b", r"\bparlando\b", r"\bcontinuando\b", r"\baggiungendo\b", r"\bassicurando\b",
    r"\bsoggiungendo\b", r"\basserendo\b", r"\bdichiarando\b", r"\btestimoniando\b",

    # imperfect indicative verbs
    r"\bmi replicava\b", r"\bmi parlava\b", r"\bmi diceva\b", r"\bmi soggiungeva\b", r"\bmi raccordava\b",

    # past simple
    r"\bdisse\b", r"\brispose\b", r"\bcontinuò\b", r"\baggiunse\b", r"\breplicò\b", r"\bdichiarò\b", r"\bsoggiunse\b", r"\bprofferì\b",

    # first-person verb forms 
    r"\b(io )?disse\b", r"\b(io )?risposi\b", r"\b(io )?aggiunsi\b", r"\b(io )?replicai\b", r"\bio non aggiunsi altro\b",
    r"\bet io andai rispondendo\b", r"\bandai rispondendo\b", r"\bandai portando\b",

    # passive and indirect reporting
    r"\bfu proferito\b", r"\bfu detto\b", r"\bfu risposto\b", r"\bil profferì con qualche sorriso\b",
    r"\bil che mi è sta\[to\] detto anco da altri\b", r"\bet a me lo diceva in confidenza\b",

    # edge continuation or extension
    r"\bet poi soggiunse\b", r"\bet aggiunse\b", r"\bet continuò\b",
    r"\bet io risposi\b", r"\bio replicai\b", r"\bio risposi\b",

    # contextual gerunds
    r"\bvedendo ch’io\b", r"\bet essendo\b", r"\bet essere\b",
    r"\bdoppo haver\b.*?\bdiscorso\b",
]
pattern = re.compile("|".join(direct_speech_patterns), re.IGNORECASE)

# MAIN
records = []
seen_hashes = set()

for filename in os.listdir(folder_path):
    if filename.endswith(".docx") and not filename.startswith("~$"):
        file_path = os.path.join(folder_path, filename)
        doc = Document(file_path)
        full_text = " ".join([clean_text(p.text) for p in doc.paragraphs if p.text.strip()])
        if full_text:
            text_hash = get_hash(full_text)
            if text_hash not in seen_hashes:
                seen_hashes.add(text_hash)
                sentences = simple_sent_tokenize(full_text)
                prev1 = False
                prev2 = False
                temp_records = []
                for sent in sentences:
                    fixed_sent = clean_text(sent)
                    fixed_sent_lower = fixed_sent.lower()
                    is_reported = bool(pattern.search(fixed_sent_lower))
                    prev_reported = prev1 or prev2
                    temp_records.append({"sentence": fixed_sent,"is_reported": is_reported,"previous_reported": prev_reported})
                    prev2 = prev1
                    prev1 = is_reported
                # adding next_reported
                for i in range(len(temp_records)):
                    next1 = temp_records[i + 1]["is_reported"] if i + 1 < len(temp_records) else False
                    temp_records[i]["next_reported"] = next1
                records.extend(temp_records)

df = pd.DataFrame(records)
df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

In [5]:
# getting only reported speech document
reported_df = df[df["is_reported"] == True]
reported_df.to_csv("only_reported_sentences.csv", index=False, encoding="utf-8-sig")